<a href="https://colab.research.google.com/github/BenjamimOliveira/CN_TP1/blob/main/script/Script2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP1 - CN


## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import tensorflow.keras.optimizers as optimizers
import time
import zipfile
import random

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Load data sources

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

start_time = time.time()

# Copy data (.zip file)
!cp -r 'drive/MyDrive/UMinho/CN/dataset.zip' 'dataset.zip'

# Extract .zip file
with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset_extr')

print("%s minutos" % ((time.time() - start_time)))

Mounted at /content/drive
38.19596600532532 minutos


## Data Loading

In [ ]:
train_dir = '/content/dataset_extr/dataset/train'
test_dir = '/content/dataset_extr/dataset/test'
valid_dir = '/content/dataset_extr/dataset/valid'

imgDataGenerator = False

if imgDataGenerator:
  train_dtgen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
  test_dtgen = ImageDataGenerator(rescale = 1./255)
  valid_dtgen = ImageDataGenerator(rescale = 1./255)

  train_ds = train_dtgen.flow_from_directory(train_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
  test_ds = test_dtgen.flow_from_directory(test_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
  valid_ds = valid_dtgen.flow_from_directory(valid_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
else:
  train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      train_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  test_ds = tf.keras.preprocessing.image_dataset_from_directory(
      test_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
      valid_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  
  class_names = train_ds.class_names

  AUTOTUNE = tf.data.AUTOTUNE

  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)
  test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Algoritmo Genético

In [ ]:
##
# Formato do gene [nParesConv2DMaxPooling(entre 1 e 3), learningRate, Momentum, Nesterov, batchSize, Epochs]
#
##

learningRate = [1, 0.1, 0.01, 0.001, 0.0001]
batchSize = [8, 128]
epochs = [4, 20]
num_classes = 250
poolSize = 8

### Gene Aleatório

In [ ]:
def random_element():
  gene = []
  
  # -- nPares Conv2D/MaxPooling
  gene.append(random.randint(1,3))

  # -- Learning rate
  gene.append(random.randint(0,4))

  # -- Momentum
  gene.append(random.randint(0, 9)/10)

  # -- Nesterov
  gene.append(random.randint(0,1))

  # -- BatchSize
  gene.append(random.randint(batchSize[0],batchSize[1]))

  # -- Epochs
  gene.append(random.randint(epochs[0], epochs[1]))

  return gene

### População Aleatória/Inicial

In [ ]:
def random_pool_generator(poolSize):
  pool = []

  for x in range(poolSize):
    pool.append(random_element())

  return pool

### Selecionar "pais" da próxima geração

In [ ]:
def select_mating_pool(poolSize):
  pass

### Crossover dos "pais" selecionados

In [ ]:
def crossover():
  pass

### Mutação de uma geração

In [ ]:
def mutation():
  pass

### Gerador de CNN (segundo o gene)

In [ ]:
def model_generator(nPares):
  mod = []
  mod.append(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(64, 64, 3)))
  if a >= 1: 
    mod.append(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  if a >= 2:
    mod.append(layers.Conv2D(32, 3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  if a >= 3:
    mod.append(layers.Conv2D(64, 3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  mod.append(layers.Flatten())
  mod.append(layers.Dense(128, activation='relu'))
  mod.append(layers.Dense(num_classes))
  model = Sequential(mod)
  return model

### Conversor de gene em acurácia (treino da cnn com o gene)

In [ ]:
def gene_converter(gene):
  # -- MODELO
  model = model_generator(gene[0])
  # -- COMPILAR MODELO
  opt = optimizers.SGD(learning_rate=0.01, momentum=0.9)
  # Experimentar loss='sparse_categorical_crossentropy'
  if imgDataGenerator:
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  else:
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile(optimizer='adam',
                loss=loss,
                metrics=['accuracy'])
  # -- TREINAR MODELO
  epochs=1
  start_time = time.time()
  with tf.device('/device:GPU:0'):
    history = model.fit(
      train_ds,
      validation_data=valid_ds,
      epochs=epochs
    )
  print("%s minutos" % ((time.time() - start_time)/60))
  # -- AVALIAR MODELO
  results = model.evaluate(test_ds)
  return results

### Algoritmo Final

In [ ]:
# -- guardar accuracy dos parents para evitar treinar de novo
pool = random_pool_generator(poolSize)
print(pool)

maximiza = []

for gene in pool:
  maximiza.append(gene_converter(gene=gene))

In [ ]:
for x in maximiza:
  print(x)